<a href="https://colab.research.google.com/github/guiabbehusen/Projeto-ML-1/blob/main/Models/redes_neurais_recorrentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offset1)* (freq1 * 10 + 10))
  series += 0.2 * np.sin((time - offset1)* (freq1 * 20 + 20))
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)

  return series[..., np.newaxis].astype(np.float32)



In [3]:
import numpy as np
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]


In [4]:
y_pred = X_valid[:, -1]

In [5]:
from tensorflow import keras

In [6]:
import tensorflow as tf

In [7]:
mse = tf.keras.metrics.MeanSquaredError()

In [8]:
mse.update_state(y_valid, y_pred)
mse_result = mse.result().numpy()
print(mse_result)

0.01994354


In [9]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [11]:
series = generate_time_series(1, n_steps + 10)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
  y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
  X = np.concatenate([X, y_pred_one], axis=1)
y_pred = X[:, n_steps:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [12]:
print(y_pred)

[[[-0.29344398]
  [-0.1311822 ]
  [-0.00917017]
  [ 0.01825061]
  [ 0.24110225]
  [-0.10140331]
  [-0.19561537]
  [-0.32875645]
  [-0.4572664 ]
  [-0.5262949 ]]]


In [13]:
print(y_new)

[[[-0.6439351 ]
  [-0.4938202 ]
  [-0.3624675 ]
  [-0.00377995]
  [ 0.25028187]
  [ 0.45621657]
  [ 0.6020418 ]
  [ 0.5429638 ]
  [ 0.48208272]
  [ 0.3593096 ]]]


In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
y_new.shape

(1, 10, 1)

In [16]:
y_pred.shape

(1, 10, 1)

In [17]:
mse = mean_squared_error(y_new.reshape(-1), y_pred.reshape(-1))
print(mse)

0.3753504157066345


In [18]:
series = generate_time_series(10000, n_steps+10)
X_train, Y_train = series[:7000, :n_steps], series[7000:, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
Y_pred = model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step


In [21]:
mse = mean_squared_error(y_new.reshape(-1), Y_pred.reshape(-1))

In [22]:
mse

0.41323351860046387

In [23]:
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 +1):
  Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[7000:]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [24]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
def last_step(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [26]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)

In [27]:
model.compile(loss="mse", optimizer=optimizer, metrics=[last_step])


In [28]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])